<a href="https://colab.research.google.com/github/JeromeNickson/Quantum-Portfolio-Optimization/blob/main/notebooks/markowitz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
# Install dependencies
!pip install yfinance cvxpy

# Import libraries
import numpy as np
import pandas as pd
import yfinance as yf
import cvxpy as cp
import matplotlib.pyplot as plt

# Step 1: Download stock price data
tickers = ["AAPL", "MSFT", "GOOGL", "AMZN", "META"]  # Example tech stocks
data = yf.download(tickers, start="2022-01-01", end="2023-01-01")["Close"]

# Step 2: Calculate returns
returns = data.pct_change().dropna()

# Step 3: Calculate expected returns & covariance matrix
mu = returns.mean().values  # expected return
Sigma = returns.cov().values  # covariance matrix
n = len(tickers)

# Step 4: Define optimization variables
w = cp.Variable(n)  # portfolio weights

# Portfolio return and risk
portfolio_return = cp.matmul(mu, w)   # scalar value
portfolio_risk = cp.quad_form(w, Sigma)

# Step 5: Optimization problem (minimize risk for target return)
target_return = returns.mean().mean()  # you can adjust this
prob = cp.Problem(cp.Minimize(portfolio_risk),
                  [cp.sum(w) == 1, w >= 0, portfolio_return >= target_return])

# Solve
prob.solve()

print("Solver Status:", prob.status)

if prob.status == "optimal":
    weights = w.value
    print("Optimal Portfolio Weights:")
    for i, t in enumerate(tickers):
        print(f"{t}: {weights[i]:.4f}")

    print("\nExpected Portfolio Return:", portfolio_return.value)
    print("Expected Portfolio Risk (Variance):", portfolio_risk.value)
else:
    print("No feasible solution found.")


/tmp/ipython-input-2113820250.py:13: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(tickers, start="2022-01-01", end="2023-01-01")["Close"]
[*********************100%***********************]  5 of 5 completed

Solver Status: optimal
Optimal Portfolio Weights:
AAPL: 0.4623
MSFT: -0.0000
GOOGL: 0.0431
AMZN: 0.0000
META: 0.4947

Expected Portfolio Return: -0.001087628961916544
Expected Portfolio Risk (Variance): 0.00045664749526197945
